In [1]:
! pip install -q langchain-core langchain-community langchain-google-genai duckduckgo-search wikipedia arxiv

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.7 MB/s eta 0:00:00


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, Tool
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from google.colab import userdata
import os

In [3]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [4]:
search_tool = Tool(
    name="DuckDuckGo Search",
    func=DuckDuckGoSearchRun().run,
    description="User this tool to perform web search using DuckDuckGo",
)

In [5]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [6]:
tool = [search_tool]

In [7]:
agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)

<ipython-input-7-aac89eea2271>:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)


In [10]:
response = agent.run("Indian Budget news")
response



> Entering new AgentExecutor chain...
Thought: I need to search for recent news related to the Indian budget.  I'll use DuckDuckGo to find relevant articles.

Action: DuckDuckGo Search
Action Input: "Indian Budget 2023 news"
Observation: "The focus of the budget is taking everyone together on an inclusive path," Sitharaman said, adding that the government is aiming for a fiscal deficit of 4.4% of India's gross domestic ... Here are five key takeaways from India's union budget: ... could be limited since a tiny fraction of Indians pay direct taxes. In 2023, 1.6% of Indians (22.4 million people) actually paid income ... In conclusion, the 2025-2026 Budget is a comprehensive blueprint for India's growth, balancing fiscal prudence with inclusive develop­ment. By focusing on agriculture, MSMEs, green energy, and exports, the government aims to create a robust foundation for long-term economic stability. India's Finance Minister is expected to present a budget with tax cuts for the middle 

'The Indian Union Budget 2023, presented by Finance Minister Nirmala Sitharaman, focused on inclusive development, infrastructure, green growth, and youth empowerment under the theme "Saptarishi."  Key highlights included a significant capital outlay for Indian Railways, tax relief for the middle class, and emphasis on initiatives like Aadhaar, Cowin, and UPI.  The budget aimed to balance fiscal prudence with economic growth.'

# Wikipedia Search Tool

In [11]:
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

In [12]:
search_tool = Tool(
    name="Wikipedia Search",
    func=WikipediaAPIWrapper().run,
    description="User this tool to perform search in wikipedia",
)

In [13]:
tool = [search_tool]

In [14]:
agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)

In [15]:
response = agent.run("OpenAI")
response



> Entering new AgentExecutor chain...
Thought: I need to find information about OpenAI.  A Wikipedia search should provide a good overview.
Action: Wikipedia Search
Action Input: OpenAI
Observation: Page: OpenAI
Summary: OpenAI is an American artificial intelligence (AI) research organization founded in December 2015 and headquartered in San Francisco, California. Its stated mission is to develop "safe and beneficial" artificial general intelligence (AGI), which it defines as "highly autonomous systems that outperform humans at most economically valuable work". As a leading organization in the ongoing AI boom, OpenAI is known for the GPT family of large language models, the DALL-E series of text-to-image models, and a text-to-video model named Sora. Its release of ChatGPT in November 2022 has been credited with catalyzing widespread interest in generative AI.
The organization consists of the non-profit OpenAI, Inc., registered in Delaware, and its for-profit subsidiary introduced in 

"OpenAI is an American artificial intelligence research company founded in 2015.  Its mission is to develop safe and beneficial artificial general intelligence.  It's known for models like GPT, DALL-E, and Sora, and has faced challenges including lawsuits and leadership changes.  Microsoft is a major investor."

# Sqlite Database

In [16]:
import sqlite3

In [17]:
conn = sqlite3.connect("example.db")

In [18]:
cursor = conn.cursor()

In [19]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
  id INTEGER PRIMARY KEY,
  name TEXT,
  quantity INTEGER
)
""")

cursor.executemany("""
INSERT INTO inventory (name, quantity)
VALUES (?, ?)
""", [
    ("Apple", 10),
    ("Banana", 20),
    ("Orange", 15)
])

conn.commit()
conn.close()

In [20]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

In [21]:
db = SQLDatabase.from_uri("sqlite:///example.db")

In [22]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [23]:
tool = toolkit.get_tools()

In [24]:
agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)

In [25]:
response = agent.run("How many rows are there in the inventory table?")
response



> Entering new AgentExecutor chain...
Thought: I need to get the number of rows in the inventory table.  I'll first check if the table exists using `sql_db_list_tables`. Then I'll construct a SQL query to count the rows.

Action: sql_db_list_tables
Action Input: ""
Observation: inventory
Thought:Thought: The inventory table exists. Now I'll create a SQL query to count the rows.

Action: sql_db_query_checker
Action Input: SELECT COUNT(*) FROM inventory;
Observation: SELECT COUNT(*) FROM inventory;
Thought:Thought: The query checker simply echoed the query.  This means it's syntactically correct. I'll now execute it.

Action: sql_db_query
Action Input: SELECT COUNT(*) FROM inventory;
Observation: [(3,)]
Thought:Thought: I now know the final answer. The query returned (3,), indicating there are 3 rows in the inventory table.

Final Answer: 3

> Finished chain.


'3'

In [26]:
response = agent.run("Quanitity of product Orange")
response



> Entering new AgentExecutor chain...
Thought: I need to query the database to find the quantity of the product "Orange".  I first need to know the table names to check which table contains the product information.
Action: sql_db_list_tables
Action Input: 
Observation: inventory
Thought:Thought: Now that I know the table name, I need to check the schema to see which columns are relevant.
Action: sql_db_schema
Action Input: inventory
Observation: 
CREATE TABLE inventory (
	id INTEGER, 
	name TEXT, 
	quantity INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from inventory table:
id	name	quantity
1	Apple	10
2	Banana	20
3	Orange	15
*/
Thought:Thought: I can now construct a SQL query to get the quantity of "Orange".
Action: sql_db_query_checker
Action Input: SELECT quantity FROM inventory WHERE name = 'Orange';
Observation: SELECT quantity FROM inventory WHERE name = 'Orange';
Thought:Action: sql_db_query
Action Input: SELECT quantity FROM inventory WHERE name = 'Orange';
Observation: [(15,)]
Tho

'15'

# Arxiv

In [27]:
from langchain_community.tools.arxiv.tool import ArxivQueryRun

In [28]:
arxiv_tool = Tool(
    name="arxiv",
    func=ArxivQueryRun().run,
    description="Use this tool to search papers on arxiv. Input should be a query.",
)

In [29]:
tool = [arxiv_tool]

In [30]:
agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)

In [31]:
response = agent.run("Attention is all you need")
response



> Entering new AgentExecutor chain...
Thought: The question "Attention is all you need" is likely referring to the famous paper that introduced the Transformer architecture.  I should search for this paper on arXiv.

Action: arxiv
Action Input: "Attention is all you need"
Observation: Published: 2024-07-22
Title: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models
Authors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini
Summary: The inference demand for LLMs has skyrocketed in recent months, and serving
models with low latencies remains challenging due to the quadratic input length
complexity of the attention layers. In this work, we investigate the effect of
dropping MLP and attention layers at inference time on the performance of
Llama-v2 models. We find that dropping dreeper attention layers only marginally
decreases performance but leads to the best speedups alongside dropping entire
layers. For example, removing 33

'The question "Attention is all you need" refers to a seminal paper introducing the Transformer architecture, authored by Vaswani et al.  While I couldn\'t directly retrieve the paper\'s abstract through the provided arXiv tool, numerous subsequent papers cite it as foundational to the field of Transformer networks.'

In [32]:
response = agent.run("Research Paper on Retrieval Augmentation Generation. Give summary of one")
response



> Entering new AgentExecutor chain...
Thought: I need to search for research papers on Retrieval Augmentation Generation (RAG) on arXiv.
Action: arxiv
Action Input: "Retrieval Augmentation Generation"
Observation: Published: 2024-10-30
Title: R^2AG: Incorporating Retrieval Information into Retrieval Augmented Generation
Authors: Fuda Ye, Shuangyin Li, Yongqi Zhang, Lei Chen
Summary: Retrieval augmented generation (RAG) has been applied in many scenarios to
augment large language models (LLMs) with external documents provided by
retrievers. However, a semantic gap exists between LLMs and retrievers due to
differences in their training objectives and architectures. This misalignment
forces LLMs to passively accept the documents provided by the retrievers,
leading to incomprehension in the generation process, where the LLMs are
burdened with the task of distinguishing these documents using their inherent
knowledge. This paper proposes R$^2$AG, a novel enhanced RAG framework to fill
this

'The paper "R^2AG: Incorporating Retrieval Information into Retrieval Augmented Generation" addresses the semantic gap between Large Language Models (LLMs) and retrievers in Retrieval Augmented Generation (RAG).  This gap causes LLMs to passively use retrieved documents, hindering comprehension.  R^2AG improves RAG by incorporating retrieval information using a novel R^2-Former and a retrieval-aware prompting strategy. This allows the LLM to better understand and utilize the retrieved information, particularly beneficial in low-source scenarios where the LLM and retriever are frozen. Experiments across five datasets show R^2AG\'s effectiveness, robustness, and efficiency.  The retrieval information acts as an anchor, bridging the semantic gap and improving generation.'

# File Agent

In [33]:
from langchain_community.tools.file_management import ReadFileTool

In [34]:
read_file_tool = Tool(
    name="read_file",
    func=ReadFileTool().run,
    description="Use this tool to read a file. Input should be the file path.",
)

In [35]:
tool = [read_file_tool]

In [36]:
agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)

In [38]:
file_name = "001ssb.txt"

response = agent.run(f"Summary of {file_name}")
response



> Entering new AgentExecutor chain...
Thought: I need to read the content of the file 001ssb.txt to create a summary.
Action: read_file
Action Input: '001ssb.txt'
Observation: Error: no such file or directory: '001ssb.txt'
Thought:Thought: I cannot access the file '001ssb.txt'.  Therefore, I cannot provide a summary.
Final Answer: I am unable to provide a summary because the file '001ssb.txt' could not be found.

> Finished chain.


"I am unable to provide a summary because the file '001ssb.txt' could not be found."